In [49]:
from carbon.helpers.cryptocompare import CryptoCompare
import pandas as pd
from matplotlib import pyplot as plt
import os
print( "{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CryptoCompare))

CryptoCompare v1.4 (01/Feb/2023)


# Crypto Compare

https://min-api.cryptocompare.com/documentation

## Selection

Please enter the pairs for the `SELECTION` data collection below

In [50]:
selection = "MIM/LTC, BNT/LINK, NMR/USD"
hourly = "DAI/USDC, USDT/USDC, DAI/USDT, USDD/USDT, MIM/USDC, wstETH/ETH, cbETH/ETH, STETH/ETH, USDC/USDT, BTC/ETH, BTC/USD, ETH/USD"
hourly_unavailable = "frxETH/ETH, rETH/ETH, ETH/sETH2"

In [51]:
pd.read_pickle("cryptocompare/SELECTION.pickle").head()

,BNT/LINK,MIM/LTC,NMR/USD
datetime,,,
2017-08-11,0.0,0.0,34.72
2017-08-12,0.0,0.0,43.90
2017-08-13,0.0,0.0,36.76
2017-08-14,0.0,0.0,34.62
2017-08-15,0.0,0.0,31.30


In [52]:
# !cp cryptocompare/SELECTION.pickle .
# !cp cryptocompare/SELECTION.pickle ../../../JupyLiteCarbonSim/content/data
# !cp cryptocompare/SELECTION.pickle ../../../BinderCarbonSimulator/Frozen/data

## SetUp

If you have a private API key, run `export CCAPIKEY=<YOURKEY>` before you launch Jupyter on the same shell. In case no key is found the free API is used. You can also provide the API key in the constructor, but this is not recommended for security reasons.

In [53]:
try:
    CC = CryptoCompare(apikeyname="CCAPIKEY")
except:
    CC = CryptoCompare(apikey=True)

print("[CryptoCompare] key digest", CC.keydigest[:4])

Can't find API key CCAPIKEY in environment variables.
Use `export CCAPIKEY=<value>` to set it BEFORE you launch Jupyter
[CryptoCompare] key digest 0000


## Coin lists

Here we populate the various coin lists that we will use to download data. They are as follows:

- `coins_for_ccy`, `ccies`: creates a table for each of the currencies 
- `coins_for_cross`: creates a single cross table
- `usdstables`: creates a single table against USD

We also define the list of items **not** to produce, `excludes`. Finally there is an `includes` list that, if present, specifies the _only_ items to be produced.

In [54]:
coins_for_ccy ="ETH, BTC, BNB, XRP, ADA, DOGE, MATIC, SOL, OKB, DOT, LTC, TRX, AVAX, ATOM"
coins_for_cross ="ETH, BTC, BNB, XRP, ADA, DOGE, MATIC, SOL, OKB, DOT, LTC, TRX, AVAX, ATOM"
usdstables = "USDT, USDC, UST, BUSD"
ccies = "USD, BTC, ETH"

#excludes = ['COINS-CROSS', 'STABLES-USD', 'COINS-USD', 'COINS-BTC', 'COINS-ETH']
includes = ["HOURLY"]

The table `dltable0` has as keys the filename, and the data is a tuple of pairs. The table `dltable` contains the final downloads, the difference being the `excludes` to avoid redownloading data that is not needed. 

**YOU MUST RESTART THE KERNEL IF YOU MAKE CHANGES TO INCLUDES OR EXCLUDES**.

In [55]:
dltable0 = {
    "COINS-CROSS": (CC.create_pairs(coins_for_cross), CC.query_dailypair),
    "STABLES-USD": (CC.create_pairs(usdstables, "USD"), CC.query_dailypair),
    **{
        f"COINS-{ccy}": (CC.create_pairs(coins_for_ccy, ccy), CC.query_dailypair)
        for ccy in CC.coinlist(ccies)
    },
    "SELECTION": (CC.coinlist(selection, aspt=True), CC.query_dailypair),
    "HOURLY": (CC.coinlist(hourly, aspt=True), CC.query_hourlypair),
}
try:
    dltable = {k:v for k, v in dltable0.items() if k in includes}
    print("restricted execution to the pairs in `includes`", includes)
except:
    try:
        dltable = {k:v for k, v in dltable0.items() if not k in excludes}
        print("the items in `excludes` were excluded", excludes)
    except:
        dltable = dltable0
        print("no `includes` or `excludes` info present; all tables are being process")
dltable0.keys(), dltable.keys()

restricted execution to the pairs in `includes` ['HOURLY']


(dict_keys(['COINS-CROSS', 'STABLES-USD', 'COINS-USD', 'COINS-BTC', 'COINS-ETH', 'SELECTION', 'HOURLY']),
 dict_keys(['HOURLY']))

## Data download

### Raw tables

In [56]:
for item, val in dltable.items():
    pairs, cc_query_f = val
    print("Downloading raw table", item, len(pairs))
    results = {
        #(fsym, tsym): CC.query_dailypair(fsym=fsym, tsym=tsym)
        (fsym, tsym): cc_query_f(fsym=fsym, tsym=tsym)
        for fsym, tsym in pairs
    }
    df = pd.concat(results, axis=1)
    df.to_pickle(CC.datafn(f"{item}-RAW.pickle"))

[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'DAI', 'tsym': 'USDC', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'USDT', 'tsym': 'USDC', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'DAI', 'tsym': 'USDT', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'USDD', 'tsym': 'USDT', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'MIM', 'tsym': 'USDC', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'wstETH', 'tsym': 'ETH', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'cbETH', 'tsym': 'ETH', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'STETH', 'tsym': 'ETH', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histohour {'fsym': 'USDC', 'tsym

In [57]:
!ls cryptocompare

COINS-BTC-RAW.pickle      COINS-USD-RAW.pickle      STABLES-USD-RAW.pickle
COINS-BTC.pickle          COINS-USD.pickle          STABLES-USD.pickle
COINS-CROSS-RAW.pickle    HOURLY-RAW.pickle         isoccy.csv
COINS-CROSS.pickle        HOURLY.pickle             query_allexchanges.pickle
COINS-ETH-RAW.pickle      SELECTION-RAW.pickle      query_coinlist.pickle
COINS-ETH.pickle          SELECTION.pickle          query_indexlist.pickle


### Reformatted tables

In [58]:
for item in dltable:
    print("Converting raw table", item)
    df = pd.read_pickle(CC.datafn(f"{item}-RAW.pickle"))
    df = CC.reformat_raw_df(df)
    df.to_pickle(CC.datafn(f"{item}.pickle"))

Converting raw table HOURLY


## Review tables

In [59]:
!ls cryptocompare

COINS-BTC-RAW.pickle      COINS-USD-RAW.pickle      STABLES-USD-RAW.pickle
COINS-BTC.pickle          COINS-USD.pickle          STABLES-USD.pickle
COINS-CROSS-RAW.pickle    HOURLY-RAW.pickle         isoccy.csv
COINS-CROSS.pickle        HOURLY.pickle             query_allexchanges.pickle
COINS-ETH-RAW.pickle      SELECTION-RAW.pickle      query_coinlist.pickle
COINS-ETH.pickle          SELECTION.pickle          query_indexlist.pickle


In [60]:
df = pd.read_pickle("cryptocompare/HOURLY.pickle")
df

,DAI/USDC,cbETH/ETH,wstETH/ETH,DAI/USDT,BTC/USD,ETH/USD,STETH/ETH,USDD/USDT,USDC/USDT,BTC/ETH,MIM/USDC,USDT/USDC
datetime,,,,,,,,,,,,
2022-12-16 11:00:00,0.9998,0.000,0.000,0.9998,17030.94,1213.35,0.9912,0.9788,1.0000,14.04,0.9922,1.0000
2022-12-16 12:00:00,0.9991,0.000,0.000,0.9990,17036.71,1213.67,0.9917,0.9865,1.0000,14.04,0.9918,1.0000
2022-12-16 13:00:00,0.9997,0.000,0.000,0.9997,16951.81,1208.26,0.9856,0.9787,1.0000,14.03,0.9905,1.0000
2022-12-16 14:00:00,0.9995,0.000,0.000,0.9990,17035.32,1214.95,0.9884,0.9800,0.9990,14.02,0.9466,1.0010
2022-12-16 15:00:00,0.9997,0.000,0.000,0.9995,16974.86,1204.05,0.9768,0.9803,1.0000,14.09,1.0020,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-09 15:00:00,0.9978,1.010,1.112,0.9990,21644.17,1533.72,0.9987,0.9997,1.0000,14.11,0.9768,1.0000
2023-03-09 16:00:00,0.9978,1.007,1.112,0.9990,21476.70,1526.36,0.9992,1.0010,1.0010,14.07,0.9943,0.9991
2023-03-09 17:00:00,0.9977,1.010,1.113,0.9990,21464.78,1528.90,0.9846,1.0000,0.9997,14.04,0.9846,1.0000
